# 概述

本文主要基于 https://github.com/nl8590687/ASRT_SpeechRecognition 介绍如何在Jupyter GPU Notebook内进行语音识别建模相关工作，详细请参考社区，此处对重点过程进行拆解，辅助用户理解Jupyter GPU Notebook使用方法以及基本的语音识别建模方法。

下文主要强调实验可复现，对原理和模型介绍，包括声学模型和统计语言模型在语音识别中的应用请参照相关文章。



# 模型介绍


模型基于CTC构建声学模型以及统计语言模型，构成完整的语音识别过程，基于CTC的深度模型如下：

```
1207 18:23:24.567357 140192177678080 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
the_input (InputLayer)          (None, 1600, 200, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1600, 200, 32 288         the_input[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1600, 200, 32 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 1600, 200, 32 9248        dropout_1[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 800, 100, 32) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 800, 100, 32) 0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 800, 100, 64) 18496       dropout_2[0][0]                  
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 800, 100, 64) 0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 800, 100, 64) 36928       dropout_3[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 400, 50, 64)  0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 400, 50, 64)  0           max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 400, 50, 128) 73856       dropout_4[0][0]                  
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 400, 50, 128) 0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 400, 50, 128) 147584      dropout_5[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 200, 25, 128) 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 200, 25, 128) 0           max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 200, 25, 128) 147584      dropout_6[0][0]                  
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 200, 25, 128) 0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 200, 25, 128) 147584      dropout_7[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 200, 25, 128) 0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 200, 25, 128) 0           max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 200, 25, 128) 147584      dropout_8[0][0]                  
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 200, 25, 128) 0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 200, 25, 128) 147584      dropout_9[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 200, 25, 128) 0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 200, 3200)    0           max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 200, 3200)    0           reshape_1[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 200, 128)     409728      dropout_10[0][0]                 
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 200, 128)     0           dense_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200, 1428)    184212      dropout_11[0][0]                 
__________________________________________________________________________________________________
Activation0 (Activation)        (None, 200, 1428)    0           dense_2[0][0]                    
__________________________________________________________________________________________________
the_labels (InputLayer)         (None, 64)           0                                            
__________________________________________________________________________________________________
input_length (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
label_length (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
ctc (Lambda)                    (None, 1)            0           Activation0[0][0]                
                                                                 the_labels[0][0]                 
                                                                 input_length[0][0]               
                                                                 label_length[0][0]               
==================================================================================================
Total params: 1,470,676
Trainable params: 1,470,676
Non-trainable params: 0
```
    

# 下载源代码

In [1]:
!git clone https://github.com/nl8590687/ASRT_SpeechRecognition.git

fatal: destination path 'ASRT_SpeechRecognition' already exists and is not an empty directory.


#  准备环境

In [2]:
!pip install tensorflow-gpu==1.14.0
!pip install python_speech_features
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.3.1
!pip install wave
!pip install matplotlib
!pip install requests

Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: 

You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://pypi.douban.com/simple
You are using pip version 19.0.1, however version 20.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.



# 下载并解压语音数据集

参考 https://github.com/nl8590687/ASRT_SpeechRecognition#data-sets-%E6%95%B0%E6%8D%AE%E9%9B%86 


下面重点下载下面2个数据集

* **清华大学THCHS30中文语音数据集**

  data_thchs30.tgz 
[OpenSLR国内镜像](<http://openslr.magicdatatech.com/resources/18/data_thchs30.tgz>)
[OpenSLR国外镜像](<http://www.openslr.org/resources/18/data_thchs30.tgz>)


* **Free ST Chinese Mandarin Corpus** 

  ST-CMDS-20170001_1-OS.tar.gz 
[OpenSLR国内镜像](<http://openslr.magicdatatech.com/resources/38/ST-CMDS-20170001_1-OS.tar.gz>)
[OpenSLR国外镜像](<http://www.openslr.org/resources/38/ST-CMDS-20170001_1-OS.tar.gz>)

准备好具备下列目录结构的文件

下载数据集，并解压，可能时间会比较久

In [ ]:
wget http://openslr.magicdatatech.com/resources/18/data_thchs30.tgz

In [ ]:
wget http://openslr.magicdatatech.com/resources/38/ST-CMDS-20170001_1-OS.tar.gz

In [3]:
!tar xvf data_thchs30.tgz && tar xvf ST-CMDS-20170001_1-OS.tar.gz

ASRT_SpeechRecognition	      data_thchs30.tgz		    tmp
ASRT_SpeechRecognition.ipynb  ST-CMDS-20170001_1-OS
data_thchs30		      ST-CMDS-20170001_1-OS.tar.gz


In [4]:
!ls -l

total 14340928
drwxr-xr-x 10 jovyan users  1867093316 Dec  7 16:56 ASRT_SpeechRecognition
-rw-r--r--  1 jovyan users       18789 Dec  7 17:56 ASRT_SpeechRecognition.ipynb
drwxr-xr-x  8 jovyan users  7981774022 Dec 30  2015 data_thchs30
-rw-r--r--  1 jovyan users  6453425169 Oct  4  2017 data_thchs30.tgz
drwxr-xr-x  2 jovyan users 12691938519 Feb  2  2018 ST-CMDS-20170001_1-OS
-rw-r--r--  1 jovyan users  8231662593 Feb  2  2018 ST-CMDS-20170001_1-OS.tar.gz
drwxr-xr-x  5 jovyan users  4270271416 Dec  7 17:53 tmp




# 训练模型



## 准备配置

先准备dataset数据配置，下面命令会准备相关数据数据，用于训练声学模型


In [ ]:
!cd ASRT_SpeechRecognition/dataset/ && \
    ln -s ./../../ST-CMDS-20170001_1-OS ST-CMDS-20170001_1-OS && \
    ln -s ./../../data_thchs30 data_thchs30

In [ ]:
!cp -r datalist/* dict.txt ASRT_SpeechRecognition/dataset/

## 启动训练

下面训练过程可能持续数小时，甚至更久，如果想加速训练过程，请参阅Easy-AI分布式并行训练方法，此处不赘述。

In [1]:
!cd ASRT_SpeechRecognition && python train_mspeech.py

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

W1207 18:23:25.551744 140192177678080 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

[*Info] Create Model Successful, Compiles Model Successful. 
[running] train epoch 0 .
[message] epoch 0 . Have train datas 0+
Epoch 1/1
2020-12-07 18:23:30.365346: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2020-12-07 18:23:30.490927: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0


# 模型优化

上述一个简单过程，可以尝试从模型设计、训练方法、数据处理等多角度对模型进行优化设计，此处不赘述。

# 测试模型


训练好的模型默认会保持在model_speech目录下，我们选择对最新的模型进行测试

主要修改 `~/ASRT_SpeechRecognition/test.py` 中下面两行，

- 我们下面以第25500步模型进行测试，模型名为`speech_model251_e_0_step_25500.model` ，故

```
ms.LoadModel(modelpath + 'm51/speech_model251_e_0_step_25500.model')
```

- 测试文件为`/home/jovyan/ST-CMDS-20170001_1-OS/20170001P00241I0052.wav`, 故

```
r = ms.RecognizeSpeech_FromFile('/home/jovyan/ST-CMDS-20170001_1-OS/20170001P00241I0052.wav')
```

In [2]:
!cd ~/ASRT_SpeechRecognition && ls model_speech/m251

speech_model251_e_0_step_10000.base.h5
speech_model251_e_0_step_10000.h5
speech_model251_e_0_step_10000.model
speech_model251_e_0_step_10000.model.base
speech_model251_e_0_step_1000.base.h5
speech_model251_e_0_step_1000.h5
speech_model251_e_0_step_1000.model
speech_model251_e_0_step_1000.model.base
speech_model251_e_0_step_10500.base.h5
speech_model251_e_0_step_10500.h5
speech_model251_e_0_step_10500.model
speech_model251_e_0_step_10500.model.base
speech_model251_e_0_step_11000.base.h5
speech_model251_e_0_step_11000.h5
speech_model251_e_0_step_11000.model
speech_model251_e_0_step_11000.model.base
speech_model251_e_0_step_11500.base.h5
speech_model251_e_0_step_11500.h5
speech_model251_e_0_step_11500.model
speech_model251_e_0_step_11500.model.base
speech_model251_e_0_step_12000.base.h5
speech_model251_e_0_step_12000.h5
speech_model251_e_0_step_12000.model
speech_model251_e_0_step_12000.model.base
speech_model251_e_0_step_12500.base.h5
speech_model251_e_0_step_12500.h5
speech_model251_e_0

In [5]:
!cd ~/ASRT_SpeechRecognition && python test.py

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2020-12-07 18:43:51.266876: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-12-07 18:43:51.687732: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561433a57020 executing computations on platform CUDA. Devices:
2020-12-07 18:43:51.687804: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): GeForce GTX 1080 Ti, Compute Capability 6.1
2020-12-07 18:43:51.687819: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (1): GeForce GTX 1080 Ti, Compute Capability 6.1
2020-12-07 18:43:51.692557: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200110000 Hz
2020-12-07 18:43:51.698835: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561433b9e8a0 executing computations on platform Host. Devices:
2020-12-07 18:43:51.698892: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


2020-12-07 18:43:54.634054: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
*[提示] 语音识别结果：
 ['wo3', 'men5', 'qun2', 'ni3', 'xiong1', 'di4', 'jian3', 'mei4', 'dou1', 'zai4', 'shuo1']
语音转文字结果：
 我们群你兄弟捡妹都在说


# 结尾

关于语音识别算法，上面只是一种demo级的算法设计过程，进行工业级算法设计仍然需要很多工作，例如，

- 大量复杂的数据工程，用于数据存储和计算，统计语言模型计算、数据预处理等，相关工作可以利用数据平台支持。

- 需要高算力系统弄支持复杂模型的训练，用于应对工业界大量的数据计算，这里可以利用算法平台大规模并行分布式训练的能力。

- 需要大量反复的实验，反复实验对比，反复的模型效果对比等等，这里可以利用算法可视化建模实验管理能力管理实验。

- 为了取得更好的业务效果，还需要结合业务场景特点，充分利用语音语言、深度学习以及业务特点，持续挖掘算法效果。